In [1]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np
from utils import *
import pickle 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import hamming_loss
from sklearn.metrics import f1_score
import time

start =time.time()

n = 20000

num_tags = 100

#load all lyric data into pandas dataframe
df = pd.read_csv('lyric_data.csv', index_col=0)

# Sometimes the API returns an error message rather than actual lyrics. This removes it
bad_song = df['lyrics'].value_counts().index[0]
df[df['lyrics'] == bad_song] = ''

# only take the ones that we have data for
df.fillna('', inplace=True)
df = df[df['lyrics'] != '']

# List of list of tags for each example
tags = [clean_tags(raw) for raw in list(df['tags'])]

# list of tuples of (tag, frequency) in desending order
tf = tag_freq(tags)

# Choose which tags to restrict model too
important_tags = [x[0] for x in tf[0:num_tags]]
important_tags = dict(zip(important_tags, range(len(important_tags))))

# maps each of the tags int 'tags' to an int index
indices = tag2index(tags, important_tags)

# Convert indices to binary vectors of tags
y = np.zeros((len(indices), num_tags))
for i, tags in enumerate(indices):
    for tag in tags:
        y[i, tag] = 1

# Build vocabulary and tokenizer
vect = CountVectorizer(max_features=n, stop_words='english')
X = vect.fit_transform(df['lyrics'])
vocab = vect.vocabulary_
tok = vect.build_analyzer()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#model =ML_KNN(5,100)
#model.fit(X_train[0:50000],y_train[0:50000])
#model.score(X_test[0:5000],y_test[0:5000])



NameError: name 'ML_KNN' is not defined

In [27]:
from scipy.spatial.distance import pdist
from sklearn.metrics.pairwise import pairwise_distances

def nearest_neighbors(X,k):
    
    D = pairwise_distances(X[0:5],X)
    
    print(D.shape)
    neighbors = D.argpartition( k , axis=1)[:,:k]
    
    return(neighbors)

start =time.time()
N = nearest_neighbors(X_train[0:100],5)
print(time.time() - start)

(5, 100)
0.003993034362792969


In [24]:
nn = NearestNeighbors(n_neighbors=5, radius=1.0)
nn.fit(X_train)

start = time.time()
N = nn.kneighbors(X_train[0:1000])
print(time.time() - start)

15.49823784828186


In [3]:
#identify prior probabilities 
#TODO: ADD SMOOTHING PARAM
H = y_train.sum(axis=0) / y_train.sum(axis=0).sum()


In [ ]:
#Posterior Probabilites

x_test = X_test[0]
    
    
c1 = np.zeros((5,100))
c2 = np.zeros((5,100))

for i in range(X_train.shape[0]):
#for i in range(5,6):

    N = nn.kneighbors(X_train[i])

    delta = y_train[N[1][0]].sum(axis=0).astype(int) -1 

    y_xi = y_train[i]
    
    mask = np.nonzero(y_xi)
    mask_neg = np.nonzero(1-y_xi)
    
    c1[ tuple(delta[mask]) , tuple(mask[0])] += 1
    c2[ tuple(delta[mask_neg]) , tuple(mask_neg[0])] += 1
    
    if i % 10 ==0:
        
        print(i,X_train.shape[0])
    
        

In [ ]:
a,b = nn.kneighbors(X_train[0:10])

In [ ]:
#rng = nn.radius_neighbors(X_test[0])
a.shape

In [ ]:
print(y_train[5])
print(np.nonzero(y_train[5]))

In [4]:
start = time.time()
N = nn.kneighbors(X_train[0:500])
print(time.time() - start)

10.639411926269531


In [9]:
N[1][2]

array([    2, 95489, 57340, 66627, 14498])

In [30]:
#Posterior Probabilites

x_test = X_test[0]
    

N = nn.kneighbors(X_train[0:500])
    
c1 = np.zeros((5+1,100))
c2 = np.zeros((5+1,100))

for i in range(500):
#for i in range(5,6):

    delta = y_train[N[1][i]].sum(axis=0).astype(int)

    y_xi = y_train[i]
    
    mask = np.nonzero(y_xi)
    mask_neg = np.nonzero(1-y_xi)
    
    c1[ tuple(delta[mask]) , tuple(mask[0])] += 1
    c2[ tuple(delta[mask_neg]) , tuple(mask_neg[0])] += 1
    
    if i % 10 ==0:
        
        print(i,X_train.shape[0])
    
        

0 150236
10 150236
20 150236
30 150236
40 150236
50 150236
60 150236
70 150236
80 150236
90 150236
100 150236
110 150236
120 150236
130 150236
140 150236
150 150236
160 150236
170 150236
180 150236
190 150236
200 150236
210 150236
220 150236
230 150236
240 150236
250 150236
260 150236
270 150236
280 150236
290 150236
300 150236
310 150236
320 150236
330 150236
340 150236
350 150236
360 150236
370 150236
380 150236
390 150236
400 150236
410 150236
420 150236
430 150236
440 150236
450 150236
460 150236
470 150236
480 150236
490 150236


In [40]:
c2[:,10]

array([ 347.,   83.,   12.,    5.,    1.,    0.])

In [41]:
E_H1 = (c1+1) / (c1+1).sum(axis=0)
E_H0 = (c2+1) / (c2+1).sum(axis=0)

In [49]:
#predict new point

x_test = X_test[0:10]
N_test = nn.kneighbors(x_test)

count = y_train[N_test[1][0]].sum(axis=0).astype(int)



In [51]:
N_test[1].shape

(10, 5)

In [22]:
E_H1[]

(5, 100)

In [44]:
E_H1[ tuple(count) , tuple(range(100))] * H

array([ 0.00290319,  0.00424264,  0.00027715,  0.00026028,  0.0003271 ,
        0.00210817,  0.00268018,  0.00029365,  0.00107769,  0.00222078,
        0.0013776 ,  0.00090018,  0.0007685 ,  0.00027432,  0.00130546,
        0.0011022 ,  0.00029298,  0.00024445,  0.00026838,  0.00131382,
        0.00151207,  0.00094386,  0.00027258,  0.00030599,  0.00052449,
        0.00242604,  0.0025678 ,  0.00019889,  0.0004467 ,  0.0006853 ,
        0.00070225,  0.00023593,  0.00468837,  0.00118532,  0.000888  ,
        0.00026298,  0.0016305 ,  0.00025244,  0.00089311,  0.00109259,
        0.00024617,  0.00102818,  0.00132836,  0.00060829,  0.00074314,
        0.00114635,  0.001023  ,  0.00107906,  0.00024894,  0.00160755,
        0.00074564,  0.00103922,  0.00021335,  0.00077296,  0.00046162,
        0.00131288,  0.00088191,  0.00147916,  0.00084925,  0.00089118,
        0.00071373,  0.00064541,  0.00091064,  0.00051242,  0.00066708,
        0.00023794,  0.00024039,  0.00054068,  0.0002954 ,  0.00

In [45]:
E_H0[ tuple(count) , tuple(range(100))] *( 1 - H )

array([ 0.00551303,  0.0049165 ,  0.53161694,  0.53783294,  0.69063076,
        0.00226601,  0.00220664,  0.00219019,  0.00218937,  0.00433436,
        0.00216744,  0.004288  ,  0.00437386,  0.76369328,  0.0021735 ,
        0.00214683,  0.74295857,  0.74334214,  0.58622473,  0.00428842,
        0.00211219,  0.00210335,  0.76571084,  0.00209651,  0.00212004,
        0.00211141,  0.00207656,  0.8061407 ,  0.00213619,  0.00213154,
        0.00411275,  0.69913595,  0.22130909,  0.00211046,  0.00211994,
        0.00208478,  0.00205081,  0.87433412,  0.00411232,  0.0020614 ,
        0.88790311,  0.0020662 ,  0.00412376,  0.00204492,  0.00207062,
        0.00415148,  0.0040579 ,  0.00208452,  0.70759455,  0.00203337,
        0.00205922,  0.0041235 ,  0.00207687,  0.00205121,  0.00206413,
        0.00409448,  0.00413727,  0.00203463,  0.00203886,  0.00412929,
        0.00205611,  0.00202286,  0.00206052,  0.00408781,  0.00206085,
        0.00204488,  0.00204105,  0.00202868,  0.00202051,  0.00

In [47]:
1*(E_H1[ tuple(count) , tuple(range(100))] * H >  E_H0[ tuple(count) , tuple(range(100))] *( 1 - H ))

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [48]:
y_test[0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [68]:
from sklearn.metrics import f1_score


class ML_KNN():
    
    def __init__(self,k,num_labels):
        
        self.nn = NearestNeighbors(n_neighbors=k, radius=1.0)
        self.k = k
        self.num_labels = num_labels
    
    def fit(self,X,y):
        
        #identify prior probabilities 
        counts = y_train.sum(axis=0)
        self.H = (counts+1) / (counts+1).sum(axis=0).sum()

        #Fit posterior probabilities
        self.nn.fit(X)
        N = self.nn.kneighbors(X)
    
        c1 = np.zeros((self.k+1,self.num_labels))
        c0 = np.zeros((self.k+1,self.num_labels))

        for i in range(500):

            delta = y_train[N[1][i]].sum(axis=0).astype(int)

            mask = np.nonzero(y[i])
            mask_neg = np.nonzero(1-y[i])

            c1[ tuple(delta[mask]) , tuple(mask[0])] += 1
            c0[ tuple(delta[mask_neg]) , tuple(mask_neg[0])] += 1

        self.E_H1 = (c1+1) / (c1+1).sum(axis=0)
        self.E_H0 = (c0+1) / (c0+1).sum(axis=0)
    
    def predict(self,X):
    
        N_test = self.nn.kneighbors(X)

        y_hat = np.zeros((X.shape[0],self.num_labels))
        
        for i in range(X.shape[0]):
        
            count = y_train[N_test[1][i]].sum(axis=0).astype(int)

            prob_hasLabel = E_H1[ tuple(count) , tuple(range(100))] * H
            prob_noLabel = E_H0[ tuple(count) , tuple(range(100))] *( 1 - H )
            
            y_hat[i] = 1*(prob_hasLabel > prob_noLabel)
            
        return y_hat
    
    def score(self,X,y):
        
        y_hat = self.predict(X)
        
        hamming = hamming_loss(y,y_hat)
        
        f = f1_score(y,y_hat,average = 'micro')
        
        return(hamming,f)

In [69]:
model =ML_KNN(5,100)

In [72]:
model.fit(X_train[0:50000],y_train[0:50000])

In [73]:
model.score(X_test[0:5000],y_test[0:5000])

(0.064528, 0.0025967602324718682)

In [2]:
X_train.shape

NameError: name 'X_train' is not defined